# Answering Exploratory Question 3: Does combining deep learning models improve accuracy?

In [ ]:
import numpy as np

In [ ]:
# Load the .npy files
X_train = np.load('RoBERTa_train_lemma_embeddings.npy')
X_test = np.load('RoBERTa_test_lemma_embeddings.npy')

# Optional: check the shape and type
print(f"Train embeddings shape: {X_train.shape}, dtype: {X_train.dtype}")
print(f"Test embeddings shape: {X_test.shape}, dtype: {X_test.dtype}")

Train embeddings shape: (3043, 768), dtype: float32
Test embeddings shape: (761, 768), dtype: float32


In [ ]:
y_train = np.load('y_train_roberta_lemma.npy')
y_test = np.load('y_test_roberta_lemma.npy')

print(f"Train output shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"Test output shape: {y_test.shape}, dtype: {y_test.dtype}")

Train output shape: (3043,), dtype: int64
Test output shape: (761,), dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize and train the logistic regression model
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# Predict on test set
y_pred = logreg.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7450722733245729

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.68      0.72       361
           1       0.74      0.80      0.77       400

    accuracy                           0.75       761
   macro avg       0.75      0.74      0.74       761
weighted avg       0.75      0.75      0.74       761


Confusion Matrix:
 [[247 114]
 [ 80 320]]


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Step 1: Initialize the SVM model
svm_model = SVC(kernel='linear', C=1.0, random_state=42)

# Step 2: Fit the model to training data
svm_model.fit(X_train, y_train)

# Step 3: Predict on test data
y_pred = svm_model.predict(X_test)

# Step 4: Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7568988173455979
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.70      0.73       361
           1       0.75      0.81      0.78       400

    accuracy                           0.76       761
   macro avg       0.76      0.75      0.75       761
weighted avg       0.76      0.76      0.76       761

Confusion Matrix:
 [[254 107]
 [ 78 322]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Step 1: Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Step 2: Fit the model to training data
rf_model.fit(X_train, y_train)

# Step 3: Predict on test data
y_pred = rf_model.predict(X_test)

# Step 4: Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7240473061760841
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.63      0.68       361
           1       0.71      0.81      0.76       400

    accuracy                           0.72       761
   macro avg       0.73      0.72      0.72       761
weighted avg       0.73      0.72      0.72       761

Confusion Matrix:
 [[227 134]
 [ 76 324]]


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Step 1: Initialize the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Step 2: Fit the model to training data
xgb_model.fit(X_train, y_train)

# Step 3: Predict on test data
y_pred = xgb_model.predict(X_test)

# Step 4: Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:18:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.7529566360052562
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.68      0.72       361
           1       0.74      0.82      0.78       400

    accuracy                           0.75       761
   macro avg       0.76      0.75      0.75       761
weighted avg       0.75      0.75      0.75       761

Confusion Matrix:
 [[246 115]
 [ 73 327]]


In [ ]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# ====================================
# 1. GPU Setup
# ====================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# ====================================
# 2. Convert NumPy arrays to PyTorch Tensors
# ====================================
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [ ]:
# ====================================
# 3. Create DataLoaders
# ====================================
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# LSTM

In [ ]:
# ====================================
# 4. LSTM Model (No Dropout)
# ====================================
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, lstm_layers=2):
        super(LSTMClassifier, self).__init__()

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            batch_first=True,
            dropout=0  # No dropout
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # Add seq_len=1 for LSTM: (batch_size, 1, input_dim)
        _, (hn, _) = self.lstm(x)
        last_hidden = hn[-1]  # Take last layer's hidden state
        logits = self.classifier(last_hidden)
        return logits

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer
# ====================================
model = LSTMClassifier(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40 #ACCORDING TO VAL LOSS
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2471 - Val Loss: 8.2897
Epoch 2/40 - Train Loss: 32.9737 - Val Loss: 8.2331
Epoch 3/40 - Train Loss: 29.9022 - Val Loss: 7.4294
Epoch 4/40 - Train Loss: 27.2256 - Val Loss: 6.5893
Epoch 5/40 - Train Loss: 26.2487 - Val Loss: 6.5032
Epoch 6/40 - Train Loss: 25.4152 - Val Loss: 6.7957
Epoch 7/40 - Train Loss: 25.6904 - Val Loss: 6.5187
Epoch 8/40 - Train Loss: 24.8074 - Val Loss: 6.2627
Epoch 9/40 - Train Loss: 24.6659 - Val Loss: 6.5993
Epoch 10/40 - Train Loss: 24.1702 - Val Loss: 6.2818
Epoch 11/40 - Train Loss: 24.0046 - Val Loss: 6.2105
Epoch 12/40 - Train Loss: 23.4935 - Val Loss: 6.2897
Epoch 13/40 - Train Loss: 23.3470 - Val Loss: 6.1218
Epoch 14/40 - Train Loss: 23.9436 - Val Loss: 6.9036
Epoch 15/40 - Train Loss: 23.3874 - Val Loss: 6.1165
Epoch 16/40 - Train Loss: 22.8026 - Val Loss: 6.0923
Epoch 17/40 - Train Loss: 22.4786 - Val Loss: 6.2441
Epoch 18/40 - Train Loss: 22.4921 - Val Loss: 6.2613
Epoch 19/40 - Train Loss: 23.1544 - Val Loss: 7.1878
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Classification Report
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))

# Confusion Matrix (numbers only)
print("🧮 Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.7543

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7364    0.7507    0.7435       361
         1.0     0.7710    0.7575    0.7642       400

    accuracy                         0.7543       761
   macro avg     0.7537    0.7541    0.7538       761
weighted avg     0.7546    0.7543    0.7544       761

🧮 Confusion Matrix:
 [[271  90]
 [ 97 303]]


In [ ]:
# ====================================
# 4. Enhanced Bidirectional-LSTM Model (without Dropout)
# ====================================
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, lstm_layers=3, bidirectional=True, dropout=0):
        super(LSTMClassifier, self).__init__()
        self.bidirectional = bidirectional

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=dropout if lstm_layers > 1 else 0  # dropout only applies if num_layers > 1
        )

        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim

        self.classifier = nn.Sequential(
            nn.Linear(lstm_output_dim, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_len=1, input_dim)
        _, (hn, _) = self.lstm(x)

        if self.bidirectional:
            # Concatenate last forward and backward hidden states
            last_hidden = torch.cat((hn[-2], hn[-1]), dim=1)
        else:
            last_hidden = hn[-1]

        logits = self.classifier(last_hidden)
        return logits

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer
# ====================================
model = LSTMClassifier(input_dim=X_train.shape[1]).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.1870 - Val Loss: 8.2351
Epoch 2/40 - Train Loss: 31.2642 - Val Loss: 7.1709
Epoch 3/40 - Train Loss: 30.2460 - Val Loss: 7.1522
Epoch 4/40 - Train Loss: 27.4655 - Val Loss: 6.6326
Epoch 5/40 - Train Loss: 26.4071 - Val Loss: 7.3475
Epoch 6/40 - Train Loss: 26.2919 - Val Loss: 6.3642
Epoch 7/40 - Train Loss: 25.3017 - Val Loss: 6.7039
Epoch 8/40 - Train Loss: 24.7842 - Val Loss: 7.3729
Epoch 9/40 - Train Loss: 24.9695 - Val Loss: 6.2500
Epoch 10/40 - Train Loss: 24.6301 - Val Loss: 6.9598
Epoch 11/40 - Train Loss: 24.1791 - Val Loss: 6.1490
Epoch 12/40 - Train Loss: 24.0524 - Val Loss: 6.2512
Epoch 13/40 - Train Loss: 23.3947 - Val Loss: 6.4233
Epoch 14/40 - Train Loss: 23.2953 - Val Loss: 6.6704
Epoch 15/40 - Train Loss: 24.0422 - Val Loss: 6.0676
Epoch 16/40 - Train Loss: 22.7252 - Val Loss: 6.4482
Epoch 17/40 - Train Loss: 22.7631 - Val Loss: 6.0920
Epoch 18/40 - Train Loss: 22.8568 - Val Loss: 6.1075
Epoch 19/40 - Train Loss: 23.0279 - Val Loss: 6.0823
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Classification Report
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))

print("\n📌 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.7661

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7215    0.8255    0.7700       361
         1.0     0.8190    0.7125    0.7620       400

    accuracy                         0.7661       761
   macro avg     0.7703    0.7690    0.7660       761
weighted avg     0.7728    0.7661    0.7658       761


📌 Confusion Matrix:
[[298  63]
 [115 285]]


In [ ]:
# ====================================
# 4. LSTM Model (Increased Complexity, No Dropout, Not Bidirectional)
# ====================================
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, lstm_layers=3):
        super(LSTMClassifier, self).__init__()

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            batch_first=True,
            dropout=0  # No dropout
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_len=1, input_dim)
        _, (hn, _) = self.lstm(x)
        last_hidden = hn[-1]  # Take last layer's hidden state
        logits = self.classifier(last_hidden)
        return logits


In [ ]:
# ====================================
# 5. Model, Loss, Optimizer
# ====================================
model = LSTMClassifier(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2420 - Val Loss: 8.3040
Epoch 2/40 - Train Loss: 33.2169 - Val Loss: 8.2947
Epoch 3/40 - Train Loss: 31.4607 - Val Loss: 7.4654
Epoch 4/40 - Train Loss: 27.8755 - Val Loss: 6.7663
Epoch 5/40 - Train Loss: 25.7994 - Val Loss: 6.6108
Epoch 6/40 - Train Loss: 25.1818 - Val Loss: 6.4646
Epoch 7/40 - Train Loss: 26.7360 - Val Loss: 8.4092
Epoch 8/40 - Train Loss: 26.2721 - Val Loss: 6.5695
Epoch 9/40 - Train Loss: 24.4999 - Val Loss: 6.3004
Epoch 10/40 - Train Loss: 25.4325 - Val Loss: 6.6056
Epoch 11/40 - Train Loss: 24.0944 - Val Loss: 6.1773
Epoch 12/40 - Train Loss: 24.3076 - Val Loss: 6.1308
Epoch 13/40 - Train Loss: 23.5895 - Val Loss: 6.2143
Epoch 14/40 - Train Loss: 23.5553 - Val Loss: 6.7992
Epoch 15/40 - Train Loss: 23.9165 - Val Loss: 6.4113
Epoch 16/40 - Train Loss: 23.4195 - Val Loss: 6.3158
Epoch 17/40 - Train Loss: 23.2769 - Val Loss: 6.1628
Epoch 18/40 - Train Loss: 23.2450 - Val Loss: 6.1482
Epoch 19/40 - Train Loss: 22.9186 - Val Loss: 6.2391
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Classification Report
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))

print("\n📌 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.7608

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7412    0.7618    0.7514       361
         1.0     0.7795    0.7600    0.7696       400

    accuracy                         0.7608       761
   macro avg     0.7604    0.7609    0.7605       761
weighted avg     0.7613    0.7608    0.7610       761


📌 Confusion Matrix:
[[275  86]
 [ 96 304]]


#Bi-GRU

In [ ]:
# ====================================
# 4. Bi-GRU Model without Dropout
# ====================================
class BiGRUClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, gru_layers=2):
        super(BiGRUClassifier, self).__init__()

        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=gru_layers,
            batch_first=True,
            dropout=0,  # No dropout
            bidirectional=True
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_len=1, input_dim)
        gru_out, _ = self.gru(x)
        last_hidden = gru_out[:, -1, :]  # Use last time step output
        logits = self.classifier(last_hidden)
        return logits

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer
# ====================================
model = BiGRUClassifier(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.1962 - Val Loss: 8.2685
Epoch 2/40 - Train Loss: 31.5790 - Val Loss: 8.1433
Epoch 3/40 - Train Loss: 27.7871 - Val Loss: 6.7042
Epoch 4/40 - Train Loss: 28.0982 - Val Loss: 7.2166
Epoch 5/40 - Train Loss: 27.2202 - Val Loss: 7.1406
Epoch 6/40 - Train Loss: 26.1921 - Val Loss: 6.4957
Epoch 7/40 - Train Loss: 25.2505 - Val Loss: 6.5684
Epoch 8/40 - Train Loss: 24.7116 - Val Loss: 6.2812
Epoch 9/40 - Train Loss: 24.3306 - Val Loss: 6.3108
Epoch 10/40 - Train Loss: 24.4255 - Val Loss: 6.1963
Epoch 11/40 - Train Loss: 23.8030 - Val Loss: 6.3291
Epoch 12/40 - Train Loss: 24.8819 - Val Loss: 6.8526
Epoch 13/40 - Train Loss: 24.9601 - Val Loss: 6.2221
Epoch 14/40 - Train Loss: 23.6838 - Val Loss: 6.0938
Epoch 15/40 - Train Loss: 23.3214 - Val Loss: 6.0522
Epoch 16/40 - Train Loss: 22.8846 - Val Loss: 6.0634
Epoch 17/40 - Train Loss: 23.4673 - Val Loss: 6.1039
Epoch 18/40 - Train Loss: 22.5001 - Val Loss: 6.5045
Epoch 19/40 - Train Loss: 22.9668 - Val Loss: 6.1175
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Metrics
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))

# Confusion Matrix
print("\n🔢 Confusion Matrix (TN, FP, FN, TP):")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.7398

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7840    0.6233    0.6944       361
         1.0     0.7131    0.8450    0.7735       400

    accuracy                         0.7398       761
   macro avg     0.7485    0.7341    0.7339       761
weighted avg     0.7467    0.7398    0.7360       761


🔢 Confusion Matrix (TN, FP, FN, TP):
[[225 136]
 [ 62 338]]


In [ ]:
# ====================================
# 4. Bi-GRU Model without Dropout (Increased Complexity)
# ====================================
class BiGRUClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, gru_layers=3):
        super(BiGRUClassifier, self).__init__()

        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=gru_layers,
            batch_first=True,
            dropout=0,  # No dropout
            bidirectional=True
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 128),  # hidden_dim*2 due to bidirectional
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_len=1, input_dim)
        gru_out, _ = self.gru(x)
        last_hidden = gru_out[:, -1, :]  # Use last time step output
        logits = self.classifier(last_hidden)
        return logits

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer
# ====================================
model = BiGRUClassifier(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2571 - Val Loss: 8.2885
Epoch 2/40 - Train Loss: 31.3505 - Val Loss: 7.1270
Epoch 3/40 - Train Loss: 27.8711 - Val Loss: 6.5767
Epoch 4/40 - Train Loss: 27.3523 - Val Loss: 7.0115
Epoch 5/40 - Train Loss: 25.9223 - Val Loss: 6.4252
Epoch 6/40 - Train Loss: 25.1090 - Val Loss: 6.2259
Epoch 7/40 - Train Loss: 24.5246 - Val Loss: 6.2705
Epoch 8/40 - Train Loss: 24.5115 - Val Loss: 6.2209
Epoch 9/40 - Train Loss: 23.6601 - Val Loss: 6.6234
Epoch 10/40 - Train Loss: 24.0375 - Val Loss: 6.3709
Epoch 11/40 - Train Loss: 24.6684 - Val Loss: 7.4220
Epoch 12/40 - Train Loss: 24.5052 - Val Loss: 6.0901
Epoch 13/40 - Train Loss: 23.5185 - Val Loss: 6.2987
Epoch 14/40 - Train Loss: 23.5091 - Val Loss: 6.4686
Epoch 15/40 - Train Loss: 23.1917 - Val Loss: 6.0797
Epoch 16/40 - Train Loss: 22.6149 - Val Loss: 6.4618
Epoch 17/40 - Train Loss: 22.4721 - Val Loss: 6.0429
Epoch 18/40 - Train Loss: 23.1963 - Val Loss: 6.2599
Epoch 19/40 - Train Loss: 22.9265 - Val Loss: 6.5824
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Metrics
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))

# Confusion Matrix
print("\n🔢 Confusion Matrix (TN, FP, FN, TP):")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.7516

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7240    0.7701    0.7463       361
         1.0     0.7798    0.7350    0.7568       400

    accuracy                         0.7516       761
   macro avg     0.7519    0.7525    0.7515       761
weighted avg     0.7533    0.7516    0.7518       761


🔢 Confusion Matrix (TN, FP, FN, TP):
[[278  83]
 [106 294]]


In [ ]:
# ====================================
# 4. Bi-GRU Model without Dropout (Even bigger Complexity)
# ====================================
class BiGRUClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, gru_layers=4):
        super(BiGRUClassifier, self).__init__()

        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=gru_layers,
            batch_first=True,
            dropout=0,
            bidirectional=True
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_len=1, input_dim)
        gru_out, _ = self.gru(x)
        last_hidden = gru_out[:, -1, :]  # (batch_size, hidden_dim*2)
        logits = self.classifier(last_hidden)
        return logits

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer
# ====================================
model = BiGRUClassifier(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2185 - Val Loss: 8.3070
Epoch 2/40 - Train Loss: 30.5597 - Val Loss: 6.8050
Epoch 3/40 - Train Loss: 27.9359 - Val Loss: 7.5180
Epoch 4/40 - Train Loss: 27.5013 - Val Loss: 6.4529
Epoch 5/40 - Train Loss: 25.6038 - Val Loss: 6.2682
Epoch 6/40 - Train Loss: 27.0144 - Val Loss: 6.7566
Epoch 7/40 - Train Loss: 25.4237 - Val Loss: 6.2983
Epoch 8/40 - Train Loss: 26.1289 - Val Loss: 6.7835
Epoch 9/40 - Train Loss: 25.0547 - Val Loss: 6.1895
Epoch 10/40 - Train Loss: 23.8801 - Val Loss: 6.3427
Epoch 11/40 - Train Loss: 23.7844 - Val Loss: 6.2680
Epoch 12/40 - Train Loss: 23.9370 - Val Loss: 6.5553
Epoch 13/40 - Train Loss: 24.8791 - Val Loss: 6.1053
Epoch 14/40 - Train Loss: 24.4110 - Val Loss: 6.7071
Epoch 15/40 - Train Loss: 23.6762 - Val Loss: 6.2000
Epoch 16/40 - Train Loss: 24.0292 - Val Loss: 6.7128
Epoch 17/40 - Train Loss: 23.7449 - Val Loss: 6.3697
Epoch 18/40 - Train Loss: 23.5695 - Val Loss: 6.0979
Epoch 19/40 - Train Loss: 23.2463 - Val Loss: 6.0674
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Metrics
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))

# Confusion Matrix
print("\n🔢 Confusion Matrix (TN, FP, FN, TP):")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.7530

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7874    0.6565    0.7160       361
         1.0     0.7304    0.8400    0.7814       400

    accuracy                         0.7530       761
   macro avg     0.7589    0.7483    0.7487       761
weighted avg     0.7574    0.7530    0.7504       761


🔢 Confusion Matrix (TN, FP, FN, TP):
[[237 124]
 [ 64 336]]


# CNN

In [ ]:
# ====================================
# 4. Deep CNN Model (Dropout Removed)
# ====================================
class DeepCNN(nn.Module):
    def __init__(self, input_dim, num_filters=64):
        super(DeepCNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters * 2, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        # Final output size after pooling: 768 → 384 → 192
        flattened_dim = num_filters * 2 * 192

        self.fc1 = nn.Linear(flattened_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, 1, 768)
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = DeepCNN(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop (No Early Stopping)
# ====================================
epochs = 100

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/100 - Train Loss: 33.2842 - Val Loss: 8.3369
Epoch 2/100 - Train Loss: 33.2644 - Val Loss: 8.3058
Epoch 3/100 - Train Loss: 33.2404 - Val Loss: 8.3012
Epoch 4/100 - Train Loss: 33.2296 - Val Loss: 8.3075
Epoch 5/100 - Train Loss: 33.2416 - Val Loss: 8.3019
Epoch 6/100 - Train Loss: 33.2083 - Val Loss: 8.3006
Epoch 7/100 - Train Loss: 33.2382 - Val Loss: 8.3024
Epoch 8/100 - Train Loss: 33.2157 - Val Loss: 8.2987
Epoch 9/100 - Train Loss: 33.2237 - Val Loss: 8.3045
Epoch 10/100 - Train Loss: 33.2424 - Val Loss: 8.3021
Epoch 11/100 - Train Loss: 33.2111 - Val Loss: 8.3022
Epoch 12/100 - Train Loss: 33.2081 - Val Loss: 8.3007
Epoch 13/100 - Train Loss: 33.2075 - Val Loss: 8.2992
Epoch 14/100 - Train Loss: 33.1922 - Val Loss: 8.2950
Epoch 15/100 - Train Loss: 33.1547 - Val Loss: 8.2729
Epoch 16/100 - Train Loss: 32.9946 - Val Loss: 8.2350
Epoch 17/100 - Train Loss: 33.2357 - Val Loss: 8.3039
Epoch 18/100 - Train Loss: 33.2316 - Val Loss: 8.3021
Epoch 19/100 - Train Loss: 33.2101 - 

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Classification Report
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
cm = confusion_matrix(all_labels, all_preds)
print("\n🔢 Confusion Matrix:")
print(cm)


✅ Test Accuracy: 0.7477

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7649    0.6759    0.7176       361
         1.0     0.7353    0.8125    0.7720       400

    accuracy                         0.7477       761
   macro avg     0.7501    0.7442    0.7448       761
weighted avg     0.7493    0.7477    0.7462       761


🔢 Confusion Matrix:
[[244 117]
 [ 75 325]]


In [ ]:
# ====================================
# 4. Deep More Complex CNN Model (Dropout Removed)
# ====================================
class ComplexCNN(nn.Module):
    def __init__(self, input_dim, num_filters=128):
        super(ComplexCNN, self).__init__()

        self.conv1 = nn.Conv1d(1, num_filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(num_filters)
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(num_filters, num_filters * 2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(num_filters * 2)
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.conv3 = nn.Conv1d(num_filters * 2, num_filters * 4, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(num_filters * 4)
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        # 768 → 384 → 192 → 96
        self.flattened_dim = num_filters * 4 * 96

        self.fc1 = nn.Linear(self.flattened_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = x.unsqueeze(1)

        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = DeepCNN(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop (No Early Stopping)
# ====================================
epochs = 100

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/100 - Train Loss: 33.2855 - Val Loss: 8.3038
Epoch 2/100 - Train Loss: 33.2103 - Val Loss: 8.3016
Epoch 3/100 - Train Loss: 33.2075 - Val Loss: 8.3009
Epoch 4/100 - Train Loss: 33.2176 - Val Loss: 8.3010
Epoch 5/100 - Train Loss: 33.2219 - Val Loss: 8.3023
Epoch 6/100 - Train Loss: 33.2130 - Val Loss: 8.3005
Epoch 7/100 - Train Loss: 33.2149 - Val Loss: 8.3026
Epoch 8/100 - Train Loss: 33.2243 - Val Loss: 8.3004
Epoch 9/100 - Train Loss: 33.1992 - Val Loss: 8.2969
Epoch 10/100 - Train Loss: 33.1866 - Val Loss: 8.2842
Epoch 11/100 - Train Loss: 33.1552 - Val Loss: 8.2932
Epoch 12/100 - Train Loss: 33.1430 - Val Loss: 8.2729
Epoch 13/100 - Train Loss: 32.8834 - Val Loss: 8.4158
Epoch 14/100 - Train Loss: 32.4891 - Val Loss: 8.5977
Epoch 15/100 - Train Loss: 31.7114 - Val Loss: 7.5740
Epoch 16/100 - Train Loss: 30.2332 - Val Loss: 7.3369
Epoch 17/100 - Train Loss: 29.9016 - Val Loss: 7.4932
Epoch 18/100 - Train Loss: 28.7181 - Val Loss: 7.0954
Epoch 19/100 - Train Loss: 31.1472 - 

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Classification Report
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
cm = confusion_matrix(all_labels, all_preds)
print("\n🔢 Confusion Matrix:")
print(cm)


✅ Test Accuracy: 0.7240

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7649    0.6039    0.6749       361
         1.0     0.6996    0.8325    0.7603       400

    accuracy                         0.7240       761
   macro avg     0.7322    0.7182    0.7176       761
weighted avg     0.7306    0.7240    0.7198       761


🔢 Confusion Matrix:
[[218 143]
 [ 67 333]]


In [ ]:
# ====================================
# 4. Deep Most Complex CNN Model (Dropout Removed)
# ====================================
class ComplexCNN(nn.Module):
    def __init__(self, input_dim, num_filters=128):
        super(ComplexCNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(in_channels=num_filters, out_channels=num_filters * 2, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.conv3 = nn.Conv1d(in_channels=num_filters * 2, out_channels=num_filters * 4, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        self.conv4 = nn.Conv1d(in_channels=num_filters * 4, out_channels=num_filters * 8, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool1d(kernel_size=2)

        # Final pooled size after 4 maxpools: 768 → 384 → 192 → 96 → 48
        self.flattened_dim = num_filters * 8 * 48  # 1024 * 48 = 49,152

        # Updated classifier with 4 layers: 256 → 128 → 64 → 1
        self.fc1 = nn.Linear(self.flattened_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, 1, 768)

        x = self.pool1(torch.relu(self.conv1(x)))  # (batch_size, 128, 384)
        x = self.pool2(torch.relu(self.conv2(x)))  # (batch_size, 256, 192)
        x = self.pool3(torch.relu(self.conv3(x)))  # (batch_size, 512, 96)
        x = self.pool4(torch.relu(self.conv4(x)))  # (batch_size, 1024, 48)

        x = x.view(x.size(0), -1)  # Flatten → (batch_size, 1024 * 48)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return self.fc4(x)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = DeepCNN(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop (No Early Stopping)
# ====================================
epochs = 100

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/100 - Train Loss: 33.3095 - Val Loss: 8.3010
Epoch 2/100 - Train Loss: 33.2177 - Val Loss: 8.3069
Epoch 3/100 - Train Loss: 33.2415 - Val Loss: 8.2995
Epoch 4/100 - Train Loss: 33.2267 - Val Loss: 8.3004
Epoch 5/100 - Train Loss: 33.2279 - Val Loss: 8.3007
Epoch 6/100 - Train Loss: 33.2203 - Val Loss: 8.2993
Epoch 7/100 - Train Loss: 33.2233 - Val Loss: 8.3002
Epoch 8/100 - Train Loss: 33.2197 - Val Loss: 8.2959
Epoch 9/100 - Train Loss: 33.1828 - Val Loss: 8.2908
Epoch 10/100 - Train Loss: 33.1829 - Val Loss: 8.2787
Epoch 11/100 - Train Loss: 33.1234 - Val Loss: 8.2403
Epoch 12/100 - Train Loss: 32.9684 - Val Loss: 8.1559
Epoch 13/100 - Train Loss: 32.6443 - Val Loss: 8.1617
Epoch 14/100 - Train Loss: 32.4537 - Val Loss: 7.9735
Epoch 15/100 - Train Loss: 32.2128 - Val Loss: 7.9621
Epoch 16/100 - Train Loss: 31.7016 - Val Loss: 7.9367
Epoch 17/100 - Train Loss: 30.7342 - Val Loss: 7.5321
Epoch 18/100 - Train Loss: 30.1551 - Val Loss: 7.3799
Epoch 19/100 - Train Loss: 29.3764 - 

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Classification Report
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
cm = confusion_matrix(all_labels, all_preds)
print("\n🔢 Confusion Matrix:")
print(cm)


✅ Test Accuracy: 0.7490

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7457    0.7147    0.7298       361
         1.0     0.7518    0.7800    0.7656       400

    accuracy                         0.7490       761
   macro avg     0.7487    0.7473    0.7477       761
weighted avg     0.7489    0.7490    0.7487       761


🔢 Confusion Matrix:
[[258 103]
 [ 88 312]]


# Bi-GRU_Bi-LSTM

In [ ]:
# ====================================
# 4. Parallel BiGRU_BiLSTM Model
# ====================================
class BiGRU_BiLSTM_Parallel(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, gru_layers=3, lstm_layers=3, dropout=0):
        super(BiGRU_BiLSTM_Parallel, self).__init__()

        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=gru_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0
        )

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if lstm_layers > 1 else 0
        )

        # New 4-layer classifier: 256 → 128 → 64 → 1
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 4, 256),
            nn.ReLU(),
            # nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            # nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.ReLU(),
            # nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, 1, input_dim)

        # Bi-GRU
        gru_out, _ = self.gru(x)
        gru_last = gru_out[:, -1, :]  # (batch_size, hidden_dim*2)

        # Bi-LSTM
        _, (hn, _) = self.lstm(x)
        lstm_last = torch.cat((hn[-2], hn[-1]), dim=1)  # (batch_size, hidden_dim*2)

        # Concatenate features
        combined = torch.cat((gru_last, lstm_last), dim=1)  # (batch_size, hidden_dim*4)
        return self.classifier(combined)


In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = BiGRU_BiLSTM_Parallel(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2304 - Val Loss: 8.2995
Epoch 2/40 - Train Loss: 31.5951 - Val Loss: 7.2203
Epoch 3/40 - Train Loss: 29.0437 - Val Loss: 6.7606
Epoch 4/40 - Train Loss: 28.0518 - Val Loss: 7.1685
Epoch 5/40 - Train Loss: 26.3176 - Val Loss: 6.7287
Epoch 6/40 - Train Loss: 25.2643 - Val Loss: 6.7744
Epoch 7/40 - Train Loss: 24.8976 - Val Loss: 6.3516
Epoch 8/40 - Train Loss: 24.4320 - Val Loss: 6.2208
Epoch 9/40 - Train Loss: 23.9904 - Val Loss: 6.2430
Epoch 10/40 - Train Loss: 24.5398 - Val Loss: 6.2858
Epoch 11/40 - Train Loss: 24.6053 - Val Loss: 6.1375
Epoch 12/40 - Train Loss: 23.3206 - Val Loss: 6.1044
Epoch 13/40 - Train Loss: 22.9929 - Val Loss: 6.3819
Epoch 14/40 - Train Loss: 23.4977 - Val Loss: 6.3444
Epoch 15/40 - Train Loss: 24.0822 - Val Loss: 6.8483
Epoch 16/40 - Train Loss: 23.9227 - Val Loss: 6.0586
Epoch 17/40 - Train Loss: 22.8826 - Val Loss: 6.3635
Epoch 18/40 - Train Loss: 22.2282 - Val Loss: 6.1599
Epoch 19/40 - Train Loss: 22.4037 - Val Loss: 6.0863
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Classification Report
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
cm = confusion_matrix(all_labels, all_preds)
print("\n🔢 Confusion Matrix:")
print(cm)


✅ Test Accuracy: 0.7622

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7586    0.7313    0.7447       361
         1.0     0.7651    0.7900    0.7774       400

    accuracy                         0.7622       761
   macro avg     0.7619    0.7607    0.7610       761
weighted avg     0.7620    0.7622    0.7619       761


🔢 Confusion Matrix:
[[264  97]
 [ 84 316]]


In [ ]:
# ====================================
# 4. Sequential BiGRU_BiLSTM Model
# ====================================
class BiGRU_BiLSTM_Sequential(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, lstm_layers=3, gru_layers=3, dropout=0):
        super(BiGRU_BiLSTM_Sequential, self).__init__()

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if lstm_layers > 1 else 0
        )

        self.gru = nn.GRU(
            input_size=hidden_dim * 2,
            hidden_size=hidden_dim,
            num_layers=gru_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0
        )

        # Updated classifier: 256 → 128 → 64 → 1
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, 1, input_dim)

        # Bi-LSTM
        lstm_out, _ = self.lstm(x)  # (batch_size, 1, hidden_dim*2)

        # Bi-GRU
        gru_out, _ = self.gru(lstm_out)  # (batch_size, 1, hidden_dim*2)
        last_hidden = gru_out[:, -1, :]  # final time step

        return self.classifier(last_hidden)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = BiGRU_BiLSTM_Sequential(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2896 - Val Loss: 8.3014
Epoch 2/40 - Train Loss: 32.0881 - Val Loss: 8.0753
Epoch 3/40 - Train Loss: 28.9337 - Val Loss: 7.3529
Epoch 4/40 - Train Loss: 27.4819 - Val Loss: 6.6251
Epoch 5/40 - Train Loss: 25.9906 - Val Loss: 6.3661
Epoch 6/40 - Train Loss: 25.3483 - Val Loss: 6.6626
Epoch 7/40 - Train Loss: 25.5725 - Val Loss: 6.3587
Epoch 8/40 - Train Loss: 24.6842 - Val Loss: 7.0272
Epoch 9/40 - Train Loss: 24.9046 - Val Loss: 6.5959
Epoch 10/40 - Train Loss: 24.2967 - Val Loss: 6.2266
Epoch 11/40 - Train Loss: 24.2360 - Val Loss: 6.0794
Epoch 12/40 - Train Loss: 23.8795 - Val Loss: 6.3349
Epoch 13/40 - Train Loss: 23.4626 - Val Loss: 6.2322
Epoch 14/40 - Train Loss: 23.6709 - Val Loss: 6.1734
Epoch 15/40 - Train Loss: 23.1576 - Val Loss: 6.2809
Epoch 16/40 - Train Loss: 23.1662 - Val Loss: 6.0638
Epoch 17/40 - Train Loss: 22.3335 - Val Loss: 6.2042
Epoch 18/40 - Train Loss: 22.7249 - Val Loss: 6.2159
Epoch 19/40 - Train Loss: 22.1104 - Val Loss: 6.0965
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

# Accuracy and Classification Report
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
cm = confusion_matrix(all_labels, all_preds)
print("\n🔢 Confusion Matrix:")
print(cm)


✅ Test Accuracy: 0.7516

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.6784    0.9058    0.7758       361
         1.0     0.8781    0.6125    0.7216       400

    accuracy                         0.7516       761
   macro avg     0.7783    0.7592    0.7487       761
weighted avg     0.7834    0.7516    0.7473       761


🔢 Confusion Matrix:
[[327  34]
 [155 245]]


# Bi-GRU_CNN

In [ ]:
# ====================================
# 4. Parallel CNN_BiGRU Model
# ====================================
class CNN_BiGRU_Parallel(nn.Module):
    def __init__(self, input_dim=768, gru_hidden_dim=256, gru_layers=3, cnn_filters=128):
        super(CNN_BiGRU_Parallel, self).__init__()

        # GRU branch
        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=gru_hidden_dim,
            num_layers=gru_layers,
            batch_first=True,
            dropout=0,
            bidirectional=True
        )

        # CNN branch
        self.conv1 = nn.Conv1d(1, cnn_filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(cnn_filters)
        self.pool1 = nn.MaxPool1d(2)  # 768 → 384

        self.conv2 = nn.Conv1d(cnn_filters, cnn_filters * 2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(cnn_filters * 2)
        self.pool2 = nn.MaxPool1d(2)  # 384 → 192

        self.conv3 = nn.Conv1d(cnn_filters * 2, cnn_filters * 4, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_filters * 4)
        self.pool3 = nn.MaxPool1d(2)  # 192 → 96

        self.conv4 = nn.Conv1d(cnn_filters * 4, cnn_filters * 8, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm1d(cnn_filters * 8)
        self.pool4 = nn.MaxPool1d(2)  # 96 → 48

        self.cnn_out_dim = cnn_filters * 8 * 48  # 1024 * 48 = 49,152

        # Updated combined classifier
        self.classifier = nn.Sequential(
            nn.Linear(gru_hidden_dim * 2 + self.cnn_out_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # GRU branch
        x_gru = x.unsqueeze(1)  # (batch, 1, input_dim)
        gru_out, _ = self.gru(x_gru)
        gru_feat = gru_out[:, -1, :]  # (batch, 2 * hidden_dim)

        # CNN branch
        x_cnn = x.unsqueeze(1)  # (batch, 1, input_dim)
        x_cnn = self.pool1(torch.relu(self.bn1(self.conv1(x_cnn))))  # → 384
        x_cnn = self.pool2(torch.relu(self.bn2(self.conv2(x_cnn))))  # → 192
        x_cnn = self.pool3(torch.relu(self.bn3(self.conv3(x_cnn))))  # → 96
        x_cnn = self.pool4(torch.relu(self.bn4(self.conv4(x_cnn))))  # → 48
        cnn_feat = x_cnn.view(x_cnn.size(0), -1)  # Flatten

        # Concatenate features
        combined = torch.cat((gru_feat, cnn_feat), dim=1)
        return self.classifier(combined)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = CNN_BiGRU_Parallel(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40 # ONLY FOR THIS CASE
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 34.4042 - Val Loss: 8.3038
Epoch 2/40 - Train Loss: 33.4290 - Val Loss: 8.2682
Epoch 3/40 - Train Loss: 30.5878 - Val Loss: 7.2945
Epoch 4/40 - Train Loss: 27.8898 - Val Loss: 6.8401
Epoch 5/40 - Train Loss: 26.4114 - Val Loss: 6.4651
Epoch 6/40 - Train Loss: 26.2243 - Val Loss: 6.4404
Epoch 7/40 - Train Loss: 25.3284 - Val Loss: 6.6064
Epoch 8/40 - Train Loss: 24.7127 - Val Loss: 6.5571
Epoch 9/40 - Train Loss: 24.3219 - Val Loss: 6.2282
Epoch 10/40 - Train Loss: 23.9955 - Val Loss: 6.1382
Epoch 11/40 - Train Loss: 23.4451 - Val Loss: 6.1713
Epoch 12/40 - Train Loss: 24.4223 - Val Loss: 6.2283
Epoch 13/40 - Train Loss: 24.2688 - Val Loss: 6.6038
Epoch 14/40 - Train Loss: 23.9493 - Val Loss: 7.0729
Epoch 15/40 - Train Loss: 24.1780 - Val Loss: 6.4055
Epoch 16/40 - Train Loss: 23.4161 - Val Loss: 6.7043
Epoch 17/40 - Train Loss: 23.2478 - Val Loss: 6.3722
Epoch 18/40 - Train Loss: 22.6447 - Val Loss: 6.1274
Epoch 19/40 - Train Loss: 22.4852 - Val Loss: 6.2991
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
print("\n🔢 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))



✅ Test Accuracy: 0.7503

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7232    0.7673    0.7446       361
         1.0     0.7778    0.7350    0.7558       400

    accuracy                         0.7503       761
   macro avg     0.7505    0.7512    0.7502       761
weighted avg     0.7519    0.7503    0.7505       761


🔢 Confusion Matrix:
[[277  84]
 [106 294]]


In [ ]:
# ====================================
# 4. Sequential CNN_BiGRU Model
# ====================================
class CNN_BiGRU_Sequential(nn.Module):
    def __init__(self, input_dim=768, cnn_filters=128, gru_hidden_dim=256, gru_layers=3):
        super(CNN_BiGRU_Sequential, self).__init__()

        # 4-layer CNN stack
        self.conv1 = nn.Conv1d(1, cnn_filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(cnn_filters)
        self.pool1 = nn.MaxPool1d(2)  # 768 → 384

        self.conv2 = nn.Conv1d(cnn_filters, cnn_filters * 2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(cnn_filters * 2)
        self.pool2 = nn.MaxPool1d(2)  # 384 → 192

        self.conv3 = nn.Conv1d(cnn_filters * 2, cnn_filters * 4, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_filters * 4)
        self.pool3 = nn.MaxPool1d(2)  # 192 → 96

        self.conv4 = nn.Conv1d(cnn_filters * 4, cnn_filters * 8, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm1d(cnn_filters * 8)
        self.pool4 = nn.MaxPool1d(2)  # 96 → 48

        self.cnn_out_channels = cnn_filters * 8  # 1024
        self.cnn_seq_len = 48  # after 4 poolings
        self.gru_input_dim = self.cnn_out_channels

        # Bi-GRU
        self.gru = nn.GRU(
            input_size=self.gru_input_dim,
            hidden_size=gru_hidden_dim,
            num_layers=gru_layers,
            batch_first=True,
            dropout=0,
            bidirectional=True
        )

        # Updated classifier: 256 → 128 → 64 → 1
        self.classifier = nn.Sequential(
            nn.Linear(gru_hidden_dim * 2, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch, 1, 768)

        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        x = self.pool4(torch.relu(self.bn4(self.conv4(x))))  # (batch, 1024, 48)

        x = x.permute(0, 2, 1)  # (batch, seq_len=48, feature_dim=1024)
        gru_out, _ = self.gru(x)
        last_hidden = gru_out[:, -1, :]  # (batch, 2*hidden_dim)

        return self.classifier(last_hidden)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = CNN_BiGRU_Sequential(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2710 - Val Loss: 8.3029
Epoch 2/40 - Train Loss: 33.2614 - Val Loss: 8.3047
Epoch 3/40 - Train Loss: 33.2292 - Val Loss: 8.3027
Epoch 4/40 - Train Loss: 33.2205 - Val Loss: 8.3033
Epoch 5/40 - Train Loss: 33.2227 - Val Loss: 8.3020
Epoch 6/40 - Train Loss: 33.2293 - Val Loss: 8.3027
Epoch 7/40 - Train Loss: 33.2202 - Val Loss: 8.3020
Epoch 8/40 - Train Loss: 33.2087 - Val Loss: 8.3020
Epoch 9/40 - Train Loss: 33.2097 - Val Loss: 8.3021
Epoch 10/40 - Train Loss: 33.2161 - Val Loss: 8.3021
Epoch 11/40 - Train Loss: 33.2095 - Val Loss: 8.3020
Epoch 12/40 - Train Loss: 33.2073 - Val Loss: 8.3021
Epoch 13/40 - Train Loss: 33.2190 - Val Loss: 8.3020
Epoch 14/40 - Train Loss: 33.2067 - Val Loss: 8.3020
Epoch 15/40 - Train Loss: 33.2099 - Val Loss: 8.3020
Epoch 16/40 - Train Loss: 33.2204 - Val Loss: 8.3020
Epoch 17/40 - Train Loss: 33.2157 - Val Loss: 8.3020
Epoch 18/40 - Train Loss: 33.2114 - Val Loss: 8.3021
Epoch 19/40 - Train Loss: 33.2058 - Val Loss: 8.3020
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
print("\n🔢 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))



✅ Test Accuracy: 0.5256

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.0000    0.0000    0.0000       361
         1.0     0.5256    1.0000    0.6891       400

    accuracy                         0.5256       761
   macro avg     0.2628    0.5000    0.3445       761
weighted avg     0.2763    0.5256    0.3622       761


🔢 Confusion Matrix:
[[  0 361]
 [  0 400]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# ====================================
# 4. CNN_BiGRU_Interleaved Model
# ====================================
class CNN_BiGRU_Interleaved(nn.Module):
    def __init__(self, input_dim=768, cnn_filters=128, gru_hidden=256, gru_layers=1):
        super(CNN_BiGRU_Interleaved, self).__init__()

        # Block 1: CNN ➝ BiGRU
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, cnn_filters, kernel_size=3, padding=1),
            nn.BatchNorm1d(cnn_filters),
            nn.ReLU(),
            nn.MaxPool1d(2)  # 768 → 384
        )
        self.bi_gru1 = nn.GRU(
            input_size=cnn_filters,
            hidden_size=gru_hidden,
            num_layers=gru_layers,
            batch_first=True,
            bidirectional=True
        )

        # Block 2: CNN ➝ BiGRU
        self.conv2 = nn.Sequential(
            nn.Conv1d(gru_hidden * 2, cnn_filters * 2, kernel_size=3, padding=1),
            nn.BatchNorm1d(cnn_filters * 2),
            nn.ReLU(),
            nn.MaxPool1d(2)  # 384 → 192
        )
        self.bi_gru2 = nn.GRU(
            input_size=cnn_filters * 2,
            hidden_size=gru_hidden,
            num_layers=gru_layers,
            batch_first=True,
            bidirectional=True
        )

        # Block 3: CNN ➝ BiGRU
        self.conv3 = nn.Sequential(
            nn.Conv1d(gru_hidden * 2, cnn_filters * 4, kernel_size=3, padding=1),
            nn.BatchNorm1d(cnn_filters * 4),
            nn.ReLU(),
            nn.MaxPool1d(2)  # 192 → 96
        )
        self.bi_gru3 = nn.GRU(
            input_size=cnn_filters * 4,
            hidden_size=gru_hidden,
            num_layers=gru_layers,
            batch_first=True,
            bidirectional=True
        )

        # Final classifier: 256 → 128 → 64 → 1
        self.classifier = nn.Sequential(
            nn.Linear(gru_hidden * 2, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch, 1, 768)

        # Block 1
        x = self.conv1(x)             # (batch, cnn_filters, 384)
        x = x.permute(0, 2, 1)        # (batch, 384, cnn_filters)
        x, _ = self.bi_gru1(x)        # (batch, 384, gru_hidden*2)

        # Block 2
        x = x.permute(0, 2, 1)        # (batch, gru_hidden*2, 384)
        x = self.conv2(x)             # (batch, cnn_filters*2, 192)
        x = x.permute(0, 2, 1)        # (batch, 192, cnn_filters*2)
        x, _ = self.bi_gru2(x)        # (batch, 192, gru_hidden*2)

        # Block 3
        x = x.permute(0, 2, 1)        # (batch, gru_hidden*2, 192)
        x = self.conv3(x)             # (batch, cnn_filters*4, 96)
        x = x.permute(0, 2, 1)        # (batch, 96, cnn_filters*4)
        x, _ = self.bi_gru3(x)        # (batch, 96, gru_hidden*2)

        x = x[:, -1, :]               # Last time step → (batch, gru_hidden*2)
        return self.classifier(x)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = CNN_BiGRU_Interleaved(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40 # JUST FOR THIS
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2008 - Val Loss: 8.2996
Epoch 2/40 - Train Loss: 32.8110 - Val Loss: 8.0812
Epoch 3/40 - Train Loss: 31.8001 - Val Loss: 8.3616
Epoch 4/40 - Train Loss: 31.5223 - Val Loss: 8.2458
Epoch 5/40 - Train Loss: 31.6596 - Val Loss: 7.9130
Epoch 6/40 - Train Loss: 30.6054 - Val Loss: 7.9160
Epoch 7/40 - Train Loss: 30.6874 - Val Loss: 7.8738
Epoch 8/40 - Train Loss: 29.9703 - Val Loss: 7.8177
Epoch 9/40 - Train Loss: 29.3996 - Val Loss: 7.8848
Epoch 10/40 - Train Loss: 29.0273 - Val Loss: 7.3796
Epoch 11/40 - Train Loss: 28.5408 - Val Loss: 7.7049
Epoch 12/40 - Train Loss: 28.2048 - Val Loss: 7.3271
Epoch 13/40 - Train Loss: 27.3619 - Val Loss: 7.6130
Epoch 14/40 - Train Loss: 27.4654 - Val Loss: 6.9656
Epoch 15/40 - Train Loss: 26.0507 - Val Loss: 7.4080
Epoch 16/40 - Train Loss: 25.5515 - Val Loss: 7.1057
Epoch 17/40 - Train Loss: 24.5948 - Val Loss: 7.4424
Epoch 18/40 - Train Loss: 23.1695 - Val Loss: 9.0689
Epoch 19/40 - Train Loss: 22.2581 - Val Loss: 6.6496
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
print("\n🔢 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))



✅ Test Accuracy: 0.7227

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7404    0.6399    0.6865       361
         1.0     0.7105    0.7975    0.7515       400

    accuracy                         0.7227       761
   macro avg     0.7254    0.7187    0.7190       761
weighted avg     0.7247    0.7227    0.7206       761


🔢 Confusion Matrix:
[[231 130]
 [ 81 319]]


# CNN-LSTM

In [ ]:
# ====================================
# 4. CNN_LSTM_Parallel Model
# ====================================
class CNN_LSTM_Parallel(nn.Module):
    def __init__(self, input_dim=768, cnn_filters=128, lstm_hidden=256, lstm_layers=3, dropout=0):
        super(CNN_LSTM_Parallel, self).__init__()

        # CNN Branch
        self.conv1 = nn.Conv1d(1, cnn_filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(cnn_filters)
        self.pool1 = nn.MaxPool1d(2)  # 768 → 384

        self.conv2 = nn.Conv1d(cnn_filters, cnn_filters * 2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(cnn_filters * 2)
        self.pool2 = nn.MaxPool1d(2)  # 384 → 192

        self.conv3 = nn.Conv1d(cnn_filters * 2, cnn_filters * 4, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_filters * 4)
        self.pool3 = nn.MaxPool1d(2)  # 192 → 96

        self.conv4 = nn.Conv1d(cnn_filters * 4, cnn_filters * 8, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm1d(cnn_filters * 8)
        self.pool4 = nn.MaxPool1d(2)  # 96 → 48

        self.cnn_out_dim = cnn_filters * 8 * 48  # = 1024 * 48

        # LSTM Branch
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if lstm_layers > 1 else 0
        )

        # Updated Classifier: 256 → 128 → 64 → 1
        self.classifier = nn.Sequential(
            nn.Linear(self.cnn_out_dim + lstm_hidden * 2, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # CNN Branch
        x_cnn = x.unsqueeze(1)
        x_cnn = self.pool1(torch.relu(self.bn1(self.conv1(x_cnn))))   # → 384
        x_cnn = self.pool2(torch.relu(self.bn2(self.conv2(x_cnn))))   # → 192
        x_cnn = self.pool3(torch.relu(self.bn3(self.conv3(x_cnn))))   # → 96
        x_cnn = self.pool4(torch.relu(self.bn4(self.conv4(x_cnn))))   # → 48
        x_cnn = x_cnn.view(x_cnn.size(0), -1)  # Flatten

        # LSTM Branch
        x_lstm = x.unsqueeze(1)
        _, (hn, _) = self.lstm(x_lstm)
        x_lstm = torch.cat((hn[-2], hn[-1]), dim=1)  # Final hidden states from both directions

        # Combine
        combined = torch.cat((x_cnn, x_lstm), dim=1)
        return self.classifier(combined)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = CNN_LSTM_Parallel(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 20 # ONLY FOR THIS
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/20 - Train Loss: 33.9789 - Val Loss: 8.4127
Epoch 2/20 - Train Loss: 33.2603 - Val Loss: 8.2974
Epoch 3/20 - Train Loss: 33.4433 - Val Loss: 8.5808
Epoch 4/20 - Train Loss: 33.3177 - Val Loss: 8.2971
Epoch 5/20 - Train Loss: 33.0556 - Val Loss: 8.1449
Epoch 6/20 - Train Loss: 32.2941 - Val Loss: 7.7643
Epoch 7/20 - Train Loss: 30.4453 - Val Loss: 7.5918
Epoch 8/20 - Train Loss: 29.0451 - Val Loss: 7.2366
Epoch 9/20 - Train Loss: 27.4162 - Val Loss: 6.9538
Epoch 10/20 - Train Loss: 26.6458 - Val Loss: 6.6780
Epoch 11/20 - Train Loss: 25.3993 - Val Loss: 6.7648
Epoch 12/20 - Train Loss: 25.3052 - Val Loss: 6.8172
Epoch 13/20 - Train Loss: 24.1146 - Val Loss: 6.6368
Epoch 14/20 - Train Loss: 23.7647 - Val Loss: 7.2302
Epoch 15/20 - Train Loss: 23.0633 - Val Loss: 6.9254
Epoch 16/20 - Train Loss: 22.1138 - Val Loss: 7.4344
Epoch 17/20 - Train Loss: 21.6630 - Val Loss: 6.4995
Epoch 18/20 - Train Loss: 19.8870 - Val Loss: 7.6001
Epoch 19/20 - Train Loss: 19.6312 - Val Loss: 7.2505
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
print("\n📌 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.6965

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.8652    0.4266    0.5714       361
         1.0     0.6449    0.9400    0.7650       400

    accuracy                         0.6965       761
   macro avg     0.7551    0.6833    0.6682       761
weighted avg     0.7494    0.6965    0.6732       761


📌 Confusion Matrix:
[[154 207]
 [ 24 376]]


In [ ]:
class CNN_LSTM_Sequential(nn.Module):
    def __init__(self, input_dim=768, cnn_filters=128, lstm_hidden=256, lstm_layers=3, dropout=0):
        super(CNN_LSTM_Sequential, self).__init__()

        # 4-layer CNN stack
        self.conv1 = nn.Conv1d(1, cnn_filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(cnn_filters)
        self.pool1 = nn.MaxPool1d(2)  # 768 → 384

        self.conv2 = nn.Conv1d(cnn_filters, cnn_filters * 2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(cnn_filters * 2)
        self.pool2 = nn.MaxPool1d(2)  # 384 → 192

        self.conv3 = nn.Conv1d(cnn_filters * 2, cnn_filters * 4, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_filters * 4)
        self.pool3 = nn.MaxPool1d(2)  # 192 → 96

        self.conv4 = nn.Conv1d(cnn_filters * 4, cnn_filters * 8, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm1d(cnn_filters * 8)
        self.pool4 = nn.MaxPool1d(2)  # 96 → 48

        self.lstm_input_dim = cnn_filters * 8  # 1024
        self.lstm = nn.LSTM(
            input_size=self.lstm_input_dim,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if lstm_layers > 1 else 0
        )

        # Final Classifier: 256 → 128 → 64 → 1
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden * 2, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # [B, 1, 768]

        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))  # → 384
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))  # → 192
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))  # → 96
        x = self.pool4(torch.relu(self.bn4(self.conv4(x))))  # → 48

        x = x.permute(0, 2, 1)  # [B, 48, 1024]
        _, (hn, _) = self.lstm(x)
        x = torch.cat((hn[-2], hn[-1]), dim=1)  # [B, 512]

        return self.classifier(x)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = CNN_LSTM_Sequential(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40 # ONLY FOR THIS
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2521 - Val Loss: 8.3047
Epoch 2/40 - Train Loss: 33.2334 - Val Loss: 8.3028
Epoch 3/40 - Train Loss: 33.2197 - Val Loss: 8.3027
Epoch 4/40 - Train Loss: 33.2238 - Val Loss: 8.3028
Epoch 5/40 - Train Loss: 33.2139 - Val Loss: 8.3021
Epoch 6/40 - Train Loss: 33.2197 - Val Loss: 8.3020
Epoch 7/40 - Train Loss: 33.2147 - Val Loss: 8.3020
Epoch 8/40 - Train Loss: 33.2191 - Val Loss: 8.3020
Epoch 9/40 - Train Loss: 33.2095 - Val Loss: 8.3020
Epoch 10/40 - Train Loss: 33.2109 - Val Loss: 8.3019
Epoch 11/40 - Train Loss: 33.1998 - Val Loss: 8.3016
Epoch 12/40 - Train Loss: 33.2217 - Val Loss: 8.3023
Epoch 13/40 - Train Loss: 33.2186 - Val Loss: 8.3021
Epoch 14/40 - Train Loss: 33.2170 - Val Loss: 8.3020
Epoch 15/40 - Train Loss: 33.2134 - Val Loss: 8.3020
Epoch 16/40 - Train Loss: 33.2122 - Val Loss: 8.3021
Epoch 17/40 - Train Loss: 33.2161 - Val Loss: 8.3021
Epoch 18/40 - Train Loss: 33.2035 - Val Loss: 8.3020
Epoch 19/40 - Train Loss: 33.2080 - Val Loss: 8.3020
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
print("\n📌 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.5256

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.0000    0.0000    0.0000       361
         1.0     0.5256    1.0000    0.6891       400

    accuracy                         0.5256       761
   macro avg     0.2628    0.5000    0.3445       761
weighted avg     0.2763    0.5256    0.3622       761


📌 Confusion Matrix:
[[  0 361]
 [  0 400]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# ====================================
# 4. CNN_LSTM_Interleaved Model
# ====================================
class CNN_LSTM_Interleaved(nn.Module):
    def __init__(self, input_dim=768, cnn_filters=128, lstm_hidden=256, lstm_layers=1):
        super(CNN_LSTM_Interleaved, self).__init__()

        # Block 1: CNN ➝ LSTM
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, cnn_filters, kernel_size=3, padding=1),
            nn.BatchNorm1d(cnn_filters),
            nn.ReLU(),
            nn.MaxPool1d(2)  # 768 → 384
        )
        self.lstm1 = nn.LSTM(
            input_size=cnn_filters,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True
        )

        # Block 2: CNN ➝ LSTM
        self.conv2 = nn.Sequential(
            nn.Conv1d(lstm_hidden * 2, cnn_filters * 2, kernel_size=3, padding=1),
            nn.BatchNorm1d(cnn_filters * 2),
            nn.ReLU(),
            nn.MaxPool1d(2)  # 384 → 192
        )
        self.lstm2 = nn.LSTM(
            input_size=cnn_filters * 2,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True
        )

        # Block 3: CNN ➝ LSTM
        self.conv3 = nn.Sequential(
            nn.Conv1d(lstm_hidden * 2, cnn_filters * 4, kernel_size=3, padding=1),
            nn.BatchNorm1d(cnn_filters * 4),
            nn.ReLU(),
            nn.MaxPool1d(2)  # 192 → 96
        )
        self.lstm3 = nn.LSTM(
            input_size=cnn_filters * 4,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True
        )

        # Final classifier: 256 → 128 → 64 → 1
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden * 2, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # [B, 1, 768]

        # Block 1
        x = self.conv1(x)             # [B, 128, 384]
        x = x.permute(0, 2, 1)        # [B, 384, 128]
        x, _ = self.lstm1(x)          # [B, 384, 512]

        # Block 2
        x = x.permute(0, 2, 1)        # [B, 512, 384]
        x = self.conv2(x)             # [B, 256, 192]
        x = x.permute(0, 2, 1)        # [B, 192, 256]
        x, _ = self.lstm2(x)          # [B, 192, 512]

        # Block 3
        x = x.permute(0, 2, 1)        # [B, 512, 192]
        x = self.conv3(x)             # [B, 512, 96]
        x = x.permute(0, 2, 1)        # [B, 96, 512]
        x, _ = self.lstm3(x)          # [B, 96, 512]

        # Final time step
        x = x[:, -1, :]               # [B, 512]
        return self.classifier(x)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = CNN_LSTM_Interleaved(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40 # ONLY FOR THIS
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2103 - Val Loss: 8.3374
Epoch 2/40 - Train Loss: 32.9761 - Val Loss: 8.2596
Epoch 3/40 - Train Loss: 32.7461 - Val Loss: 8.1754
Epoch 4/40 - Train Loss: 32.7339 - Val Loss: 8.4061
Epoch 5/40 - Train Loss: 32.9776 - Val Loss: 8.1798
Epoch 6/40 - Train Loss: 32.6052 - Val Loss: 8.1765
Epoch 7/40 - Train Loss: 32.7848 - Val Loss: 8.3322
Epoch 8/40 - Train Loss: 32.7193 - Val Loss: 8.0610
Epoch 9/40 - Train Loss: 31.9152 - Val Loss: 8.1054
Epoch 10/40 - Train Loss: 31.9986 - Val Loss: 8.0480
Epoch 11/40 - Train Loss: 31.8427 - Val Loss: 8.0547
Epoch 12/40 - Train Loss: 31.2468 - Val Loss: 7.8379
Epoch 13/40 - Train Loss: 31.2627 - Val Loss: 7.9126
Epoch 14/40 - Train Loss: 30.8018 - Val Loss: 7.8513
Epoch 15/40 - Train Loss: 30.4690 - Val Loss: 7.7678
Epoch 16/40 - Train Loss: 30.1293 - Val Loss: 7.7591
Epoch 17/40 - Train Loss: 30.1292 - Val Loss: 7.7481
Epoch 18/40 - Train Loss: 29.7472 - Val Loss: 7.6699
Epoch 19/40 - Train Loss: 29.6153 - Val Loss: 7.7614
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
print("\n📌 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.6649

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.6183    0.7673    0.6848       361
         1.0     0.7316    0.5725    0.6424       400

    accuracy                         0.6649       761
   macro avg     0.6750    0.6699    0.6636       761
weighted avg     0.6779    0.6649    0.6625       761


📌 Confusion Matrix:
[[277  84]
 [171 229]]


# ANN

In [ ]:
# ====================================
# 4. ANN Model
# ====================================
class ANNClassifier(nn.Module):
    def __init__(self, input_dim):
        super(ANNClassifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        return self.classifier(x)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = ANNClassifier(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40 # ONLY FOR THIS
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.2473 - Val Loss: 8.2382
Epoch 2/40 - Train Loss: 30.4616 - Val Loss: 6.7491
Epoch 3/40 - Train Loss: 27.5800 - Val Loss: 6.6576
Epoch 4/40 - Train Loss: 26.6932 - Val Loss: 6.6085
Epoch 5/40 - Train Loss: 25.8687 - Val Loss: 6.4112
Epoch 6/40 - Train Loss: 25.9460 - Val Loss: 6.4376
Epoch 7/40 - Train Loss: 25.2993 - Val Loss: 6.5985
Epoch 8/40 - Train Loss: 24.2964 - Val Loss: 6.3229
Epoch 9/40 - Train Loss: 24.0264 - Val Loss: 6.6458
Epoch 10/40 - Train Loss: 23.9917 - Val Loss: 6.6969
Epoch 11/40 - Train Loss: 24.8373 - Val Loss: 6.2256
Epoch 12/40 - Train Loss: 23.4469 - Val Loss: 6.1462
Epoch 13/40 - Train Loss: 23.5481 - Val Loss: 6.0887
Epoch 14/40 - Train Loss: 23.9560 - Val Loss: 6.7913
Epoch 15/40 - Train Loss: 23.2264 - Val Loss: 6.1710
Epoch 16/40 - Train Loss: 22.4554 - Val Loss: 6.2177
Epoch 17/40 - Train Loss: 23.1373 - Val Loss: 5.9823
Epoch 18/40 - Train Loss: 22.4928 - Val Loss: 6.1450
Epoch 19/40 - Train Loss: 22.4776 - Val Loss: 6.2277
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
print("\n📌 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.7635

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7401    0.7729    0.7561       361
         1.0     0.7865    0.7550    0.7704       400

    accuracy                         0.7635       761
   macro avg     0.7633    0.7639    0.7633       761
weighted avg     0.7644    0.7635    0.7636       761


📌 Confusion Matrix:
[[279  82]
 [ 98 302]]


In [ ]:
# ====================================
# 4. More Complex ANN Model
# ====================================
class ANNClassifier(nn.Module):
    def __init__(self, input_dim):
        super(ANNClassifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        return self.classifier(x)

In [ ]:
# ====================================
# 5. Model, Loss, Optimizer Setup
# ====================================
model = ANNClassifier(input_dim=768).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# ====================================
# 6. Training Loop
# ====================================
epochs = 40 # ONLY FOR THIS
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # === Validation Phase ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

Epoch 1/40 - Train Loss: 33.0238 - Val Loss: 7.8168
Epoch 2/40 - Train Loss: 28.5069 - Val Loss: 7.2364
Epoch 3/40 - Train Loss: 28.0093 - Val Loss: 6.4448
Epoch 4/40 - Train Loss: 27.1042 - Val Loss: 6.7354
Epoch 5/40 - Train Loss: 25.2498 - Val Loss: 6.3083
Epoch 6/40 - Train Loss: 25.4209 - Val Loss: 6.7748
Epoch 7/40 - Train Loss: 24.7428 - Val Loss: 6.3939
Epoch 8/40 - Train Loss: 24.3863 - Val Loss: 6.2417
Epoch 9/40 - Train Loss: 24.2622 - Val Loss: 6.1913
Epoch 10/40 - Train Loss: 23.7248 - Val Loss: 6.1802
Epoch 11/40 - Train Loss: 24.1460 - Val Loss: 6.1745
Epoch 12/40 - Train Loss: 23.3026 - Val Loss: 6.2439
Epoch 13/40 - Train Loss: 23.5605 - Val Loss: 6.1849
Epoch 14/40 - Train Loss: 22.7691 - Val Loss: 6.0616
Epoch 15/40 - Train Loss: 22.4485 - Val Loss: 6.1224
Epoch 16/40 - Train Loss: 22.4688 - Val Loss: 6.3699
Epoch 17/40 - Train Loss: 22.6131 - Val Loss: 6.2875
Epoch 18/40 - Train Loss: 23.2420 - Val Loss: 7.0796
Epoch 19/40 - Train Loss: 22.1500 - Val Loss: 6.0742
Ep

In [ ]:
# ====================================
# 7. Evaluation
# ====================================
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        all_preds.extend(preds.flatten())
        all_labels.extend(y_batch.numpy().flatten())

acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc:.4f}")
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))
print("\n📌 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


✅ Test Accuracy: 0.7622

📊 Classification Report:
              precision    recall  f1-score   support

         0.0     0.7123    0.8366    0.7694       361
         1.0     0.8249    0.6950    0.7544       400

    accuracy                         0.7622       761
   macro avg     0.7686    0.7658    0.7619       761
weighted avg     0.7715    0.7622    0.7615       761


📌 Confusion Matrix:
[[302  59]
 [122 278]]


# Combining Deep Learning Models does not improve accuracy.